# Written text as operational data

Written text is one type of data

### Why people write?

 - To communicate: their thoughts, feelings, urgency, needs, information

### Why people communicate?

1. To express emotions
1. To share information
1. To enable or elicit an action
1. ...

### We will use written text for the purpose other than 
1. To experience emotion
1. To learn something the author intended us to learn
1. To do what the author intended us to do

### Instead, we will use written text to recognize who wrote it
 - By calculating and comparing word frequencies in written documents
 
See, for example, likely fictional story https://medium.com/@amuse/how-the-nsa-caught-satoshi-nakamoto-868affcef595

### Example 1. Dictionaries in python (associative arrays)

Plot the frequency distribution of words on a web page.

In [49]:
import requests, re
# re is a module for regular expressions: to detect various combinations of characters
import operator

# Start from a simple document
r = requests .get('http://eecs.utk.edu')

# What comes back includes headers and other HTTP stuff, get just the body of the response
t = r.text

# obtain words by splitting a string using as separator one or more (+) space/like characters (\s) 
wds = re.split('\s+',t)

# now populate a dictionary (wf)
wf = {}
for w in wds:
    if w in wf: wf [w] = wf [w] + 1
    else:  wf[w] = 1

# dictionaries can not be sorted, so lets get a sorted *list*        
wfs = sorted (wf .items(), key = operator .itemgetter (1), reverse=True)   

# lets just have no more than 15 words 
ml = min(len(wfs),15)
for i in range(1,ml,1):
    print (wfs[i][0]+"\t"+str(wfs[i][1]))  

<!DOCTYPE	1
PUBLIC	1
"-//IETF//DTD	1
2.0//EN">	1
<html><head>	1
<title>403	1
Forbidden</title>	1
</head><body>	1
<h1>Forbidden</h1>	1
<p>You	1
don't	1
have	1
permission	1
to	1


### Example 2

Lots of markup in the output, lets remove it --- 

use BeautifulSoup and nltk modules and practice some regular expressions.

In [50]:
import requests, re, nltk
from bs4 import BeautifulSoup
from nltk import clean_html
from collections import Counter
import operator

# we may not care about the usage of stop words
stop_words = nltk.corpus.stopwords.words('english') + [
 'ut', '\'re','.', ',', '--', '\'s', '?', ')', '(', ':', '\'',
 '\"', '-', '}', '{', '&', '|', u'\u2014' ]

# We most likely would like to remove html markup
def cleanHtml (html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    return soup .get_text()

# We also want to remove special characters, quotes, etc. from each word
def cleanWord (w):
    # r in r'[.,"\']' tells to treat \ as a regular character 
    # but we need to escape ' with \'
    # any character between the brackets [] is to be removed 
    wn = re.sub('[,"\.\'&\|:@>*;/=]', "", w)
    # get rid of numbers
    return re.sub('^[0-9\.]*$', "", wn)
       
# define a function to get text/clean/calculate frequency
def get_wf (URL):
    # first get the web page
    r = requests .get(URL)
    
    # Now clean
    # remove html markup
    t = cleanHtml (r .text) .lower()
    
    # split string into an array of words using any sequence of spaces "\s+" 
    wds = re .split('\s+',t)
    
    # remove periods, commas, etc stuck to the edges of words
    for i in range(len(wds)):
        wds [i] = cleanWord (wds [i])
    
    # If satisfied with results, lets go to the next step: calculate frequencies
    # We can write a loop to create a dictionary, but 
    # there is a special function for everything in python
    # in particular for counting frequencies (like function table() in R)
    wf = Counter (wds)
    
    # Remove stop words from the dictionary wf
    for k in stop_words:
        wf. pop(k, None)
        
    #how many regular words in the document?
    tw = 0
    for w in wf:
       tw += wf[w] 
        
    
    # Get ordered list
    wfs = sorted (wf .items(), key = operator.itemgetter(1), reverse=True)
    ml = min(len(wfs),15)

    #Reverse the list because barh plots items from the bottom
    return (wfs [ 0:ml ] [::-1], tw)
        
# Now populate two lists    
(wf_ee, tw_ee) = get_wf('http://www.gutenberg.org/ebooks/1342.txt.utf-8')
(wf_bu, tw_bu) = get_wf('http://www.gutenberg.org/ebooks/76.txt.utf-8')

In [ ]:
#Plot the results: are there striking differences in language?
import numpy as np
import pylab
import matplotlib.pyplot as plt

%matplotlib inline
def plotTwoLists (wf_ee, wf_bu, title):
    f = plt.figure (figsize=(10, 6))
    # this is painfully tedious....
    f .suptitle (title, fontsize=20)
    ax = f.add_subplot(111)
    ax .spines ['top'] .set_color ('none')
    ax .spines ['bottom'] .set_color ('none')
    ax .spines ['left'] .set_color ('none')
    ax .spines ['right'] .set_color ('none')
    ax .tick_params (labelcolor='w', top='off', bottom='off', left='off', right='off', labelsize=20)

    # Create two subplots, this is the first one
    ax1 = f .add_subplot (121)
    plt .subplots_adjust (wspace=.5)

    pos = np .arange (len(wf_ee)) 
    ax1 .tick_params (axis='both', which='major', labelsize=14)
    pylab .yticks (pos, [ x [0] for x in wf_ee ])
    ax1 .barh (range(len(wf_ee)), [ x [1] for x in wf_ee ], align='center')

    ax2 = f .add_subplot (122)
    ax2 .tick_params (axis='both', which='major', labelsize=14)
    pos = np .arange (len(wf_bu)) 
    pylab .yticks (pos, [ x [0] for x in wf_bu ])
    ax2 .barh (range (len(wf_bu)), [ x [1] for x in wf_bu ], align='center')

plotTwoLists (wf_ee, wf_bu, 'Difference between Pride and Prejudice and Huck Finn')

In [ ]:
#In case Project gutenberg is blocked you can download text to your laptop and copy to the docker container via scp
#Assuming the file name you copy is pg4680.txt here is how you change the script
# Please note the option errors='replace'
# without it python invariably runs into unicode errors
f = open ('pg4680.txt', 'r', encoding="ascii", errors='replace')
    
# What comes back includes headers and other HTTP stuff, get just the body of the response
t = f.read()

# obtain words by splitting a string using as separator one or more (+) space/like characters (\s) 
wds = re.split('\s+',t)

# now populate a dictionary (wf)
wf = {}
for w in wds:
    if w in wf: wf [w] = wf [w] + 1
    else:  wf [w] = 1

# dictionaries can not be sorted, so lets get a sorted *list*        
wfs = sorted (wf .items(), key = operator .itemgetter (1), reverse=True)   

# lets just have no more than 15 words 
ml = min(len(wfs),15)
for i in range(1,ml,1):
    print (wfs[i][0]+"\t"+str(wfs[i][1]))  

# Assignment 1

1. Compare word frequencies between two works of a single author.
1. Compare word frequencies between works of two authors.
1. Are there some words preferred by one author but used less frequently by another author?

Extra credit

1. The frequency of a specific word, e.g., "would" should follow a binomial distribution (each regular word in a document is a trial and with probability p that word is "would". The estimate for p is N("would")/N(regular word)). Do these binomial distributions for your chosen word differ significantly between books of the same author or between authors? 

Project Gutenberg is a good source of for fiction and non-fiction.

E.g below are two most popular books from Project Gutenberg:
- Pride and Prejudice at http://www.gutenberg.org/ebooks/1342.txt.utf-8
- Adventures of Huckleberry Finn at http://www.gutenberg.org/ebooks/76.txt.utf-8

In [53]:
import requests, re, nltk
#In case your text is not on Project Gutenberg but at some other URL
#http://www.fullbooks.com/Our-World-or-The-Slaveholders-Daughter2.html
# that contains 12 parts
t = ""
for i in range(2,13):
  r = requests .get('http://www.fullbooks.com/Our-World-or-The-Slaveholders-Daughter' + str(i) + '.html')
  t = t + r.text

In [54]:
len(t)

1323653

In [84]:
import requests, re, nltk
import itertools 


t = ""
r = requests.get('https://www.gutenberg.org/files/1727/1727-h/1727-h.htm')
t = t + r.text

t = cleanHtml (r .text) .lower()

wds = re .split('\s+',t)

for i in range(len(wds)):
    wds [i] = cleanWord (wds [i])
    
wf = {}
for w in wds:
    if w in wf: wf [w] = wf [w] + 1
    else:  wf[w] = 1
        
wfs = sorted (wf .items(), key = operator .itemgetter (1), reverse=True)   

    
    
    
t1 = ""
r1 = requests.get('https://www.gutenberg.org/files/2199/2199-h/2199-h.htm')
t1 = t1 + r1.text

t1 = cleanHtml (r1 .text) .lower()

wds1 = re .split('\s+',t1)

for i in range(len(wds1)):
    wds1 [i] = cleanWord (wds1 [i])
    
wf1 = {}
for w in wds1:
    if w in wf1: wf1 [w] = wf1 [w] + 1
    else:  wf1[w] = 1
        
wfs1 = sorted (wf1 .items(), key = operator .itemgetter (1), reverse=True)   

        
        

t2 = ""
r2 = requests.get('https://www.gutenberg.org/files/2701/2701-h/2701-h.htm')
t2 = t2 + r2.text

t2 = cleanHtml (r2 .text) .lower()

wds2 = re .split('\s+',t2)
for i in range(len(wds2)):
    wds2 [i] = cleanWord (wds2 [i])
    
wf2 = {}
for w in wds2:
    if w in wf2: wf2 [w] = wf2 [w] + 1
    else:  wf2[w] = 1

wfs2 = sorted (wf2 .items(), key = operator .itemgetter (1), reverse=True)   

        
print("This is the Iliad:")
out = dict(itertools.islice(wfs, 30)) 
print(out)

print("This is the Odyssey:")
out1 = dict(itertools.islice(wfs1, 30)) 
print(out1)


print("This is Moby Dick:")
out2 = dict(itertools.islice(wfs2, 30)) 
print(out2)



This is the Iliad:
{'the': 7019, 'and': 5326, 'of': 3636, 'to': 3577, 'a': 2033, 'in': 1916, 'i': 1896, 'he': 1878, 'you': 1862, 'for': 1375, 'his': 1342, 'as': 1292, 'that': 1285, 'with': 1244, 'it': 1227, 'was': 1063, 'him': 1033, 'they': 962, 'is': 959, 'on': 915, 'had': 889, 'have': 878, 'me': 866, 'all': 866, 'but': 853, 'my': 817, 'not': 791, 'them': 758, 'will': 730, 'so': 689}
This is the Odyssey:
{'the': 10153, 'and': 6674, 'of': 5760, 'to': 3397, 'he': 2887, 'his': 2540, 'in': 2294, 'a': 1850, 'him': 1830, 'you': 1791, 'with': 1737, 'that': 1454, 'for': 1395, 'as': 1393, 'i': 1280, 'was': 1220, 'they': 1135, 'it': 1131, 'on': 1102, 'them': 1029, 'from': 1005, 'but': 994, 'son': 959, 'not': 915, 'had': 892, 'their': 872, 'all': 822, 'is': 818, 'will': 800, 'by': 764}
This is Moby Dick:
{'the': 14526, 'of': 6713, 'and': 6404, 'a': 4665, 'to': 4650, 'in': 4184, 'that': 2895, 'his': 2515, 'it': 2280, 'i': 1842, 'with': 1763, 'but': 1740, 'as': 1721, 'is': 1717, 'he': 1711, 'was':

The Iliad and Odysseys' most used words are "the" and this is most likely due to the wording of the wording at the time and the translation that took place. As both of these have proper nouns for names and places so it would make sense to have the word "the" as the most common. As for Moby Dick, the most common word is "the" as well and it is evident that "the" is a common word and that the authors both use common words. Perhaps manually removing these words would bring out more unique words for better comparison. Overall, both authors, Homer and Herman Melville used similar words and would most likely use more unique words without the prepositions. Such as Herman using the word "whale" 897 times which is unique to Moby Dick while Homer uses the word "son" which is unique to the Odyssey.